In [45]:
try:
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
except:
    !pip install pandas
    !pip install BeautifulSoup

In [2]:
url = "https://phd.northeastern.edu/opportunity/"

# assign headers
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",
    "Accept": "text/html",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip",
    "DNT": "1",
    "Connection": "keep-alive",
}

In [42]:
session = requests.Session()
session.headers.update(headers)

pages = 6
# Main array declaration
images = []
url_p = []
descriptions = []
titles = []
sponsors = []
dates = []
types = []

# Start loop through the session
# Scrape the first 6 pages
for i in range(pages):
    response = session.get(f"https://phd.northeastern.edu/opportunity/page/{i}")
    soup = BeautifulSoup(response.content, "html.parser")

    mainlist = soup.find_all("li", class_="media --opportunity")

    # due to different selector methods, scraping of elements are separated thru individual try catch blocks
    for i in mainlist:
        li_cont = []

        try:
            img_src = i.img["src"]
        except:
            img_src = None

        try:
            title = i.img["alt"]
        except:
            title = None

        try:
            sponsor = i.select("li", class_="list__title")[1].text
        except:
            sponsor = None

        try:
            url_page = i.a["href"]
        except:
            url_page = None

        try:
            dd = i.find("div", class_="fs--xs text--gray-400").i.text
        except:
            dd = None

        try:
            desc = i.find("div", class_="media__copy fs--xs")
            d = desc.find_all(text=True, recursive=False)[1]
        except:
            desc = None
            d = None

        try:
            # Extract the types from ul & li, unpack to a variable as a string
            ul = i.select("ul.list--inline")[1]

            for li in ul.find_all("li"):
                li_cont.append(li.text.strip())

            l = ", ".join(str(x) for x in li_cont[1:])
        except:
            ul = None
            l = None

        # Print the output
        print(img_src, "|", title, "|", url_page, "|", sponsor, "|", l, "|", dd, "|", d)

        # Append the extracted strings to the main arrays
        images.append(img_src)
        titles.append(title)
        url_p.append(url_page)
        sponsors.append(sponsor)
        dates.append(dd)
        descriptions.append(d)
        types.append(l)

https://phd.northeastern.edu/wp-content/uploads/2022/11/Claims-Conference.png | Saul Kagan Fellowship in Advanced Shoah Studies | https://phd.northeastern.edu/opportunity/saul-kagan-fellowship-in-advanced-shoah-studies-2/ | Conference on Jewish Material Claims Against Germany | Fellowship | 12/22/2023 |  Claims Conference Kagan Fellowships are available for doctoral and post-doctoral candidates around the world conducting Holocaust-related research. Each year, the… 
https://phd.northeastern.edu/wp-content/uploads/2018/01/belfer.jpg | Harvard University – John F. Kennedy School of Government – Belfer Center for Science and International Affairs – Predoctoral and Postdoctoral Fellowships | https://phd.northeastern.edu/opportunity/harvard-university-john-f-kennedy-school-of-government-belfer-center-for-science-and-international-affairs-predoctoral-and-postdoctoral-fellowships-3/ | Harvard University | Experiential PhD Opportunities, Fellowship, Predoctoral, Postdoctoral | 12/01/2023 |  Th

#### Assign scraped content to a dataframe


In [43]:
df = pd.DataFrame(
    {
        "Title": titles,
        "URL": url_p,
        "Description": descriptions,
        "Sponsor": sponsors,
        "Type": types,
        "Deadline": dates,
        "Image Link": images,
    }
)

In [20]:
df.head()

,Title,URL,Description,Sponsor,Type,Deadline,Image Link
0,Saul Kagan Fellowship in Advanced Shoah Studies,https://phd.northeastern.edu/opportunity/saul-...,Claims Conference Kagan Fellowships are avail...,Conference on Jewish Material Claims Against G...,Fellowship,12/22/2023,https://phd.northeastern.edu/wp-content/upload...
1,Harvard University – John F. Kennedy School of...,https://phd.northeastern.edu/opportunity/harva...,The Belfer Center for Science and Internation...,Harvard University,"Experiential PhD Opportunities, Fellowship, Pr...",12/01/2023,https://phd.northeastern.edu/wp-content/upload...
2,Harvard University – John F. Kennedy School of...,https://phd.northeastern.edu/opportunity/harva...,"In the 2024-2025 academic year, we will prio...",Harvard University,"Experiential PhD Opportunities, Fellowship, Pr...",12/01/2023,https://phd.northeastern.edu/wp-content/upload...
3,Kate Neal Kinley Memorial Fellowship,https://phd.northeastern.edu/opportunity/kate-...,Are you seeking advanced study in architectur...,Illinois College of Fine & Applied Arts,"Fellowship, Predoctoral",12/01/2023,https://phd.northeastern.edu/wp-content/upload...
4,American Druze Foundation Fellowship,https://phd.northeastern.edu/opportunity/ameri...,The Georgetown University Center for Contempo...,Georgetown University,"Experiential PhD Opportunities, Fellowship, Po...",01/05/2024,https://phd.northeastern.edu/wp-content/upload...


#### Export to csv


In [44]:
df.to_csv("scraped_content.csv")